In [ ]:

from google.colab import files
import networkx as nx
import io

uploaded = files.upload()
for filename in uploaded.keys():
    gml_bytes = uploaded[filename]
    gml_io = io.BytesIO(gml_bytes)
    G = nx.read_gml(gml_io)
    print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")


In [ ]:

!pip install python-louvain


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import community as community_louvain
from networkx.algorithms.community import label_propagation_communities, girvan_newman, asyn_lpa_communities
import itertools
import ipywidgets as widgets
from IPython.display import display, clear_output

# Helper to assign colors by community
def get_community_map(communities):
    community_map = {}
    for i, com in enumerate(communities):
        for node in com:
            community_map[node] = i
    return community_map

# Store all community structures
def compute_all_communities(G):
    partition = community_louvain.best_partition(G)
    louvain = {}
    for node, com in partition.items():
        louvain.setdefault(com, []).append(node)

    girvan = next(itertools.islice(girvan_newman(G), 1, None))
    label_prop = list(label_propagation_communities(G))
    async_label = list(asyn_lpa_communities(G))
    return [louvain.values(), girvan, label_prop, async_label]

community_methods = ["Louvain", "Girvan-Newman", "Label Propagation", "Async Label Propagation"]
all_communities = compute_all_communities(G)


In [ ]:

def draw_all(communities_list, show_labels=True):
    pos = nx.spring_layout(G, seed=42)
    fig, axs = plt.subplots(2, 2, figsize=(14, 12))
    axs = axs.flatten()

    for i, communities in enumerate(communities_list):
        community_map = get_community_map(communities)
        node_color = [community_map[n] for n in G.nodes()]
        nx.draw(
            G, pos, ax=axs[i],
            with_labels=show_labels,
            node_color=node_color,
            cmap=plt.cm.tab10,
            node_size=400,
            edge_color='gray'
        )
        axs[i].set_title(community_methods[i])
    plt.tight_layout()
    plt.show()

# Interactive toggle
label_toggle = widgets.Checkbox(value=True, description='Show Node Labels')
run_button = widgets.Button(description="Redraw Communities")
output = widgets.Output()

def on_click(b):
    with output:
        clear_output(wait=True)
        draw_all(all_communities, show_labels=label_toggle.value)

run_button.on_click(on_click)
display(widgets.HBox([label_toggle, run_button]), output)

# Initial render
with output:
    draw_all(all_communities, show_labels=label_toggle.value)
